원문: Moler, Cleve B. Numerical computing with MATLAB. Society for Industrial and Applied Mathematics, 2008.

이 장에서 설명하는 것은 함수의 근을 계산하는 몇가지 기본적인 방법과 3가지 방법을 복합한 빠르고 신뢰성 있는 알고리듬인 'zeroin' 이다.

$\sqrt2$를 계산해 보자. 우리는 *구간이분법*을 사용할 것인데, 이는 일종의 체계적인 시행착오법이다.  우리는 $\sqrt2$가 1 과 2 사이 임을 알고 있다.  $x=1\frac{1}{2}$ 를 시도해 보라.  $x^2$ 가 2 보다 크기 때문에 이 $x$는 너무 크다. $x=1\frac{1}{4}$ 를 시도해 보라.  $x^2$ 가 2 보다 작기 때문에 이 $x$는 너무 작다.  이렇게 반복하다 보면, 우리의 $\sqrt2$에 대한 근사값은 다음과 같다.

$$1\frac{1}{2}, \frac{1}{4}, \frac{3}{8}, \frac{5}{16}, \frac{13}{32}, \frac{27}{64}, 
\cdots$$

아래 코드가 있으며, 반복 횟수도 센다.

In [1]:
M = 2
a = 1
b = 2
k = 0
eps = 4e-16
while (b - a) > eps:
    x = (a + b) * 0.5
    if x**2 > M:
        b = x
        print('k = %5d, b = %.16g' % (k, b))
    else:
        a = x
        print('k = %5d, a = %.16g' % (k, a))
    
    k += 1

k =     0, b = 1.5
k =     1, a = 1.25
k =     2, a = 1.375
k =     3, b = 1.4375
k =     4, a = 1.40625
k =     5, b = 1.421875
k =     6, a = 1.4140625
k =     7, b = 1.41796875
k =     8, b = 1.416015625
k =     9, b = 1.4150390625
k =    10, b = 1.41455078125
k =    11, b = 1.414306640625
k =    12, a = 1.4141845703125
k =    13, b = 1.41424560546875
k =    14, b = 1.414215087890625
k =    15, a = 1.414199829101562
k =    16, a = 1.414207458496094
k =    17, a = 1.414211273193359
k =    18, a = 1.414213180541992
k =    19, b = 1.414214134216309
k =    20, b = 1.41421365737915
k =    21, a = 1.414213418960571
k =    22, a = 1.414213538169861
k =    23, b = 1.414213597774506
k =    24, b = 1.414213567972183
k =    25, a = 1.414213553071022
k =    26, a = 1.414213560521603
k =    27, b = 1.414213564246893
k =    28, b = 1.414213562384248
k =    29, a = 1.414213561452925
k =    30, a = 1.414213561918586
k =    31, a = 1.414213562151417
k =    32, a = 1.414213562267832
k =    33, a = 1.

16진수 표시로 a 와 b 의 최종 값은 다음과 같다.

In [2]:
# https://stackoverflow.com/questions/19414847
import struct

print('a = %s' % struct.pack('d', a))
print('b = %s' % struct.pack('d', b))
print('a = %s' % a.hex())
print('b = %s' % b.hex())

a = b'\xcc;\x7ff\x9e\xa0\xf6?'
b = b'\xcd;\x7ff\x9e\xa0\xf6?'
a = 0x1.6a09e667f3bccp+0
b = 0x1.6a09e667f3bcdp+0


거의 마지막 비트 까지 같다.  우리는 실제로 $\sqrt2$를 계산하지는 않았는데, 이는 무리수이고 부동소숫점으로는 표시할 수 없다.  하지만 우리는 두 *연속된* 부동소숫점 숫자를 찾았는데, 각각 이론해의 양 쪽에 자리한다. 52단계를 거쳤는데 그 까닭은 전기전자공학회 IEEE 배정도 숫자의 유효숫자 표시 부분이 52 비트이기 때문이다.  각 단계가 간격의 길이를 대략 한 비트씩 줄인 셈이다.

간격을 절반씩 줄여나가는 것은 느리지만 확실하게 $f(x)$ 를 0으로 만드는 $x$ 를 찾는 방법으로, 여기서 $f(x)$ 는 실수값 함수로 실수 매개 변수를 받아 들인다.  단지 우리가 $f(x)$ 에 대해 가정한 것은 우리가 작성한 프로그램이 어떤 x 에 대해서라도 그 결과를 계산할 수 있다는 것이다. 우리는 또한 가정한 것은 어떤 구간 $[a, b]$ 를 알고 있는데 그 사이에서 $f(x)$ 가 부호를 바꾼다는 것이다.  만일 $f(x)$가 실은 *연속* 수학 함수라면, 점 $x_*$ 가 그 구간 어딘가에 반드시 있는데 여기서 $f(x_*)=0$ 라는 것이다.